# Types of Data

In [14]:
from spark_init import start_spark

spark = start_spark()
spark

In [15]:
df = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("../data/retail-data/by-day/2010-12-01.csv")
df.printSchema()
df.createOrReplaceTempView("dfTable")

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [35]:
# They didn't do anything with dfTable in the previous cell
spark.sql("SELECT * FROM dfTable").show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows


In [16]:
from pyspark.sql.functions import lit
df.select(lit(5), lit("five"), lit(5.0))

DataFrame[5: int, five: string, 5.0: double]

In [17]:
df.show(5)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
only showing top 5 rows


In [18]:
from pyspark.sql.functions import col
df.where(col("InvoiceNo") != 536365)\
.select("InvoiceNo", "Description")\
.show(5, False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows


In [19]:
df.where("InvoiceNo <> 536365")\
.show(5, False, False)

+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                  |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|536366   |22633    |HAND WARMER UNION JACK       |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536366   |22632    |HAND WARMER RED POLKA DOT    |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536367   |84879    |ASSORTED COLOUR BIRD ORNAMENT|32      |2010-12-01 08:34:00|1.69     |13047.0   |United Kingdom|
|536367   |22745    |POPPY'S PLAYHOUSE BEDROOM    |6       |2010-12-01 08:34:00|2.1      |13047.0   |United Kingdom|
|536367   |22748    |POPPY'S PLAYHOUSE KITCHEN    |6       |2010-12-01 08:34:00|2.1      |13047.0   |United Kingdom|
+---------+---------+-----------------------------+--------+----

In [20]:
from pyspark.sql.functions import instr
priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1
df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show()

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|   Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|   536544|      DOT|DOTCOM POSTAGE|       1|2010-12-01 14:32:00|   569.77|      NULL|United Kingdom|
|   536592|      DOT|DOTCOM POSTAGE|       1|2010-12-01 17:06:00|   607.49|      NULL|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [21]:
from pyspark.sql.functions import instr
DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") >= 1
df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter))\
.where("isExpensive")\
.select("unitPrice", "isExpensive", "StockCode", "Description").show(5)

+---------+-----------+---------+--------------+
|unitPrice|isExpensive|StockCode|   Description|
+---------+-----------+---------+--------------+
|   569.77|       true|      DOT|DOTCOM POSTAGE|
|   607.49|       true|      DOT|DOTCOM POSTAGE|
+---------+-----------+---------+--------------+



In [22]:
from pyspark.sql.functions import expr, pow
fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerId"), fabricatedQuantity.alias("realQuantity")).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows


In [23]:
df.selectExpr(
"CustomerId",
"(POWER((Quantity * UnitPrice), 2.0) + 5) as realQuantity").show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows


In [24]:
df.selectExpr(
"*").show()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|2010-12-01 08:26:00|     7.65|   17850.0|United Kingdom|
|   536365|    21730|GLASS S

In [25]:
df.selectExpr(
"InvoiceNo"
, "StockCode").distinct().show(5)

+---------+---------+
|InvoiceNo|StockCode|
+---------+---------+
|   536365|   85123A|
|   536367|    22310|
|   536368|    22914|
|   536370|    22544|
|   536373|   85123A|
+---------+---------+
only showing top 5 rows


### Working with Strings

In [26]:
from pyspark.sql.functions import monotonically_increasing_id
df.select(monotonically_increasing_id()).show(2)

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
+-----------------------------+
only showing top 2 rows


In [27]:
from pyspark.sql.functions import initcap
df.select(initcap(col("Description"))).show()

+--------------------+
|initcap(Description)|
+--------------------+
|White Hanging Hea...|
| White Metal Lantern|
|Cream Cupid Heart...|
|Knitted Union Fla...|
|Red Woolly Hottie...|
|Set 7 Babushka Ne...|
|Glass Star Froste...|
|Hand Warmer Union...|
|Hand Warmer Red P...|
|Assorted Colour B...|
|Poppy's Playhouse...|
|Poppy's Playhouse...|
|Feltcraft Princes...|
|Ivory Knitted Mug...|
|Box Of 6 Assorted...|
|Box Of Vintage Ji...|
|Box Of Vintage Al...|
|Home Building Blo...|
|Love Building Blo...|
|Recipe Box With M...|
+--------------------+
only showing top 20 rows


In [28]:
from pyspark.sql.functions import regexp_replace
regex_string = "BLACK|WHITE|RED|GREEN|BLUE"
df.select(
regexp_replace(col("Description"), regex_string, "COLOR").alias("color_clean"),
col("Description")).show(2)

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows


In [29]:
from pyspark.sql.functions import translate
df.select(translate(col("Description"), "LEET", "1337"),col("Description"))\
.show(22, False)

+-----------------------------------+-----------------------------------+
|translate(Description, LEET, 1337) |Description                        |
+-----------------------------------+-----------------------------------+
|WHI73 HANGING H3AR7 7-1IGH7 HO1D3R |WHITE HANGING HEART T-LIGHT HOLDER |
|WHI73 M37A1 1AN73RN                |WHITE METAL LANTERN                |
|CR3AM CUPID H3AR7S COA7 HANG3R     |CREAM CUPID HEARTS COAT HANGER     |
|KNI773D UNION F1AG HO7 WA73R BO7713|KNITTED UNION FLAG HOT WATER BOTTLE|
|R3D WOO11Y HO77I3 WHI73 H3AR7.     |RED WOOLLY HOTTIE WHITE HEART.     |
|S37 7 BABUSHKA N3S7ING BOX3S       |SET 7 BABUSHKA NESTING BOXES       |
|G1ASS S7AR FROS73D 7-1IGH7 HO1D3R  |GLASS STAR FROSTED T-LIGHT HOLDER  |
|HAND WARM3R UNION JACK             |HAND WARMER UNION JACK             |
|HAND WARM3R R3D PO1KA DO7          |HAND WARMER RED POLKA DOT          |
|ASSOR73D CO1OUR BIRD ORNAM3N7      |ASSORTED COLOUR BIRD ORNAMENT      |
|POPPY'S P1AYHOUS3 B3DROOM          |P

In [30]:
from pyspark.sql.functions import instr
containsBlack = instr(col("Description"), "BLACK") >= 1
containsWhite = instr(col("Description"), "WHITE") >= 1
df.withColumn("hasSimpleColor", containsBlack | containsWhite)\
.where("hasSimpleColor")\
.select("Description").show(30, False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|WOOD 2 DRAWER CABINET WHITE FINISH|
|WOOD S/3 CABINET ANT WHITE FINISH |
|WOODEN PICTURE FRAME WHITE FINISH |
|WOODEN FRAME ANTIQUE WHITE        |
|RED WOOLLY HOTTIE WHITE HEART.    |
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|WOOD 2 DRAWER CABINET WHITE FINISH|
|WOOD S/3 CABINET ANT WHITE FINISH |
|WOODEN PICTURE FRAME WHITE FINISH |
|WOODEN FRAME ANTIQUE WHITE        |
|RED WOOLLY HOTTIE WHITE HEART.    |
|JUMBO  BAG BAROQUE BLACK WHITE    |
|WHITE SPOT RED CERAMIC DRAWER KNOB|
|FELT EGG COSY WHITE RABBIT        |
|EDWARDIAN PARASOL BLACK           |
|WOOD BLACK BOARD ANT WHITE FINISH |
|WHITE LOVEBIRD LANTERN            |
|ORGANISER WOOD ANTIQUE WHITE      |
|

In [31]:
from pyspark.sql.functions import expr, locate
simpleColors = ["black", "white", "red", "green", "blue"]
def color_locator(column, color_string):\
return locate(color_string.upper(), column)\
.cast("boolean")\
.alias(f"is_{color_string}")
selectedColumns = [color_locator(df.Description, c) for c in simpleColors]
selectedColumns.append(expr("*")) # has to a be Column type
df.select(*selectedColumns).where(expr("is_blue OR is_green"))\
.select("*").show(30, False)

+--------+--------+------+--------+-------+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|is_black|is_white|is_red|is_green|is_blue|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+--------+--------+------+--------+-------+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|false   |false   |false |false   |true   |536368   |22914    |BLUE COAT RACK PARIS FASHION       |3       |2010-12-01 08:34:00|4.95     |13047.0   |United Kingdom|
|false   |false   |false |true    |false  |536370   |22726    |ALARM CLOCK BAKELIKE GREEN         |12      |2010-12-01 08:45:00|3.75     |12583.0   |France        |
|false   |false   |false |false   |true   |536378   |84997C   |BLUE 3 PIECE POLKADOT CUTLERY SET  |6       |2010-12-01 09:37:00|3.75     |14688.0   |United Kingdom|
|false   |

### Working with Dates and Timestamps

In [32]:
from pyspark.sql.functions import current_date, current_timestamp
dateDF = spark.range(10)\
.withColumn("today", current_date())\
.withColumn("now", current_timestamp())
dateDF.createOrReplaceTempView("dateTable")
dateDF.printSchema()
dateDF.show(5, False) # was testing to see if timestamp changes after 500k rows

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)

+---+----------+--------------------------+
|id |today     |now                       |
+---+----------+--------------------------+
|0  |2025-12-01|2025-12-01 22:39:59.430801|
|1  |2025-12-01|2025-12-01 22:39:59.430801|
|2  |2025-12-01|2025-12-01 22:39:59.430801|
|3  |2025-12-01|2025-12-01 22:39:59.430801|
|4  |2025-12-01|2025-12-01 22:39:59.430801|
+---+----------+--------------------------+
only showing top 5 rows


In [ ]:
# again, they didn't do anything with dateTable in the previous cell
spark.sql("SELECT * FROM dateTable").show(5)


+---+----------+--------------------+
| id|     today|                 now|
+---+----------+--------------------+
|  0|2025-12-01|2025-12-01 22:46:...|
|  1|2025-12-01|2025-12-01 22:46:...|
|  2|2025-12-01|2025-12-01 22:46:...|
|  3|2025-12-01|2025-12-01 22:46:...|
|  4|2025-12-01|2025-12-01 22:46:...|
+---+----------+--------------------+
only showing top 5 rows


In [33]:
from pyspark.sql.functions import date_add, date_sub, col
dateDF.select("today", date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)

+----------+------------------+------------------+
|     today|date_sub(today, 5)|date_add(today, 5)|
+----------+------------------+------------------+
|2025-12-01|        2025-11-26|        2025-12-06|
+----------+------------------+------------------+
only showing top 1 row
